In [0]:
#Reading the csv file
e_comm= spark.read.csv("dbfs:/data.csv",header=True)

I have an E-Commerce dataset in which I decided to predict the number of stock that will be buyed by the customer, which is the "Quantity" column present in the dataset, and this will be my target variable, the dataset also has columns which includes invoice number, customer id, decription of stock etc.

The dataset was pretty balanced However, I needed to cleanup a bit, remove null values in order to make a model which is best for my dataset.As in this dataset my target column was "Quantity", further I made 3 pipelines to get the data: 2 pipelines for the numerical and categorical features and 3rd pipeline for the target column.

Further when you'll proceed with notebook you'll see I made 3 models in order to test which is best for my data, I selected the Decision Tree Classifier,Random Forest and NaiveBayes Classifier.For the training purpose I took the sample of 200k which I thought initially was a bad choice as databricks was taking a lot of time :) any ways it was worth it my model got trained well.

RESULTS:

For the Decision Tree Classifier I got the accuracy of 60.4% which was the highest among all.The other two models Random Forest and Naive bayes classifier showed the accuracy of 17.7% and 33% respectively.

In [0]:
#description of the columns in dataframe
e_comm.show

Out[2]: <bound method DataFrame.show of DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]>

In [0]:
#counting the number of rows in the dataset
e_comm.count()

Out[3]: 541909

In [0]:
#To take a look at data how it is stored in form of columns and rows
e_comm.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [0]:
#statistical summary of the dataframe in form of pandas
e_comm.summary().toPandas()

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,541909,541909,540455,541909,541909,541909,406829,541909
1,mean,559965.752026781,27623.240210938104,20713.0,9.55224954743324,None,4.6111136260897085,15287.690570239585,None
2,stddev,13428.417280796779,16799.737628427658,None,218.08115785023438,None,96.75985306117963,1713.6003033215982,None
3,min,536365,10002,4 PURPLE FLOCK DINNER CANDLES,-1,1/10/2011 10:04,-11062.06,12346,Australia
4,25%,547903.0,21929.0,20713.0,1.0,None,1.25,13952.0,None
5,50%,560650.0,22569.0,20713.0,3.0,None,2.08,15152.0,None
6,75%,571840.0,23165.0,20713.0,10.0,None,4.13,16791.0,None
7,max,C581569,m,wrongly sold sets,992,9/9/2011 9:52,99.96,18287,Unspecified


In [0]:
#Converting some columns to int and float from strings
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType
from pyspark.sql.functions import *
e_comm = e_comm.withColumn("Quantity",e_comm.Quantity.cast('int'))
e_comm = e_comm.withColumn("UnitPrice",e_comm.UnitPrice.cast('float'))
e_comm = e_comm.withColumn("CustomerID",e_comm.CustomerID.cast('float'))


In [0]:
#Dataframe with changed columns datatypes
e_comm.show

Out[7]: <bound method DataFrame.show of DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: float, CustomerID: float, Country: string]>

In [0]:
#Making sure only numbers greater than zero must be in these columns
e_comm = e_comm[(e_comm['Quantity']>0)]
e_comm = e_comm[(e_comm['UnitPrice']>0)]

In [0]:
#creating one column named total sales for the estimation
e_comm=e_comm.withColumn("Total_Sales",e_comm.Quantity*e_comm.UnitPrice)
e_comm.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Total_Sales|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|   17850.0|United Kingdom|  15.299999|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|      20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|   17850.0|United Kingdom|       22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|      20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|      20.34|
+---------+---------+--------------------+--------+--------------+---------+----

In [0]:
#including only the necessary columns from the dataset
necessary_cols = ['InvoiceNo','CustomerID', 'InvoiceDate','Quantity','UnitPrice','Total_Sales']
e_comm = e_comm[necessary_cols]

In [0]:
#To check the values of dataset that if we have nessecarry columns or not.
e_comm.show(5)

+---------+----------+--------------+--------+---------+-----------+
|InvoiceNo|CustomerID|   InvoiceDate|Quantity|UnitPrice|Total_Sales|
+---------+----------+--------------+--------+---------+-----------+
|   536365|   17850.0|12/1/2010 8:26|       6|     2.55|  15.299999|
|   536365|   17850.0|12/1/2010 8:26|       6|     3.39|      20.34|
|   536365|   17850.0|12/1/2010 8:26|       8|     2.75|       22.0|
|   536365|   17850.0|12/1/2010 8:26|       6|     3.39|      20.34|
|   536365|   17850.0|12/1/2010 8:26|       6|     3.39|      20.34|
+---------+----------+--------------+--------+---------+-----------+
only showing top 5 rows



In [0]:
#Here I wanted to check the datatype of each column 
e_comm.dtypes

Out[13]: [('InvoiceNo', 'string'),
 ('CustomerID', 'float'),
 ('InvoiceDate', 'string'),
 ('Quantity', 'int'),
 ('UnitPrice', 'float'),
 ('Total_Sales', 'float')]

In [0]:
#finding the number of null values in each column
e_comm.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in e_comm.columns]
   ).show()

+---------+----------+-----------+--------+---------+-----------+
|InvoiceNo|CustomerID|InvoiceDate|Quantity|UnitPrice|Total_Sales|
+---------+----------+-----------+--------+---------+-----------+
|        0|    132220|          0|       0|        0|          0|
+---------+----------+-----------+--------+---------+-----------+



In [0]:
#Since Customer Id only has null values so I'll drop the rows with null values 
#Since I cannot replace customer Id(it's unique) with statistical mean or median, so I've got to drop it.
e_comm=e_comm.na.drop()

In [0]:
#As you can see here there are no null values as I've dropped the rows with null values.
e_comm.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in e_comm.columns]
   ).show()

+---------+----------+-----------+--------+---------+-----------+
|InvoiceNo|CustomerID|InvoiceDate|Quantity|UnitPrice|Total_Sales|
+---------+----------+-----------+--------+---------+-----------+
|        0|         0|          0|       0|        0|          0|
+---------+----------+-----------+--------+---------+-----------+



In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.regression import LinearRegression

In [0]:
#pipeline for categorical columns
pipeline1 = Pipeline(stages=[
    StringIndexer(inputCol="InvoiceNo", outputCol="InvoiceNoIndex"),
    OneHotEncoder(inputCol="InvoiceNoIndex", outputCol="InvoiceNoOnehot"),
    
    StringIndexer(inputCol="InvoiceDate", outputCol="InvoiceDateIndex"),
    OneHotEncoder(inputCol="InvoiceDateIndex", outputCol="InvoiceDateOnehot"),
    
    VectorAssembler(
    inputCols=["InvoiceNoOnehot","InvoiceDateOnehot"],
  
    outputCol="categorical_features")])
    
    

In [0]:
#pipeline for the numerical values
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.feature import StandardScaler

pipeline2 = Pipeline(stages=[
    VectorAssembler(
        inputCols=["CustomerID","Total_Sales","UnitPrice"],
        outputCol="predefined_features"
    ),
    StandardScaler(
    inputCol="predefined_features",outputCol="numerical_column"
    )
    
    
    
])

In [0]:
#Pipeline for the target

pipeline3 = Pipeline(stages=[
    
        
    
    StringIndexer(
        inputCol = 'Quantity', outputCol = 'labelIndex'
    )
    
    
    

])

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier

#this is the final pipeline which inlcudes all the pipelines I made in order to transform the data.
final_pipeline = Pipeline(stages=[
    pipeline1,pipeline2, pipeline3, 
    VectorAssembler(inputCols=["numerical_column", "categorical_features"], outputCol="final_features"),
    
])

In [0]:
#data in the pipeline is transformed
transformed_data = final_pipeline.fit(e_comm).transform(e_comm)

In [0]:
#I took the sample from the original dataset to train my model
sample_ecomm=transformed_data.limit(200000)

In [0]:
# I splitted my data in  train and test used Seed= 2018 as everytime I run the block the splitting b/w train and test is fixed.
train, test = sample_ecomm.randomSplit([0.8, 0.2],seed=2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 159836
Test Dataset Count: 40164


RANDOM FOREST MODEL

In [0]:
#importing the random forest model
random_forest = RandomForestClassifier(featuresCol = 'final_features', labelCol = 'labelIndex')
random_forest_Model = random_forest.fit(train)
random_forest_Model_train = random_forest_Model.transform(train)
predictions = random_forest_Model.transform(test)

In [0]:
#Made use of MulticlassClassificationEvaluator class in python to check the accuracy of the model.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eva = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = eva.evaluate(predictions)
print("Test Accuracy",accuracy)

Test Accuracy 0.17782093416990338


We have Accuracy of 17.7% with RandomForest

In [0]:
#Importing the decision tree model
from pyspark.ml.classification import DecisionTreeClassifier

DECISION TREE MODEL

In [0]:
#Decision Tree Classifier
d_tree = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="final_features")

In [0]:
#Training data on Decision Tree Model
d_tree_Model = d_tree.fit(train)
d_tree_Model_train = d_tree_Model.transform(train)
predictions = d_tree_Model.transform(test)

In [0]:
#Made use of MulticlassClassificationEvaluator class in python to check the accuracy of the model.

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eva = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = eva.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.604746 


We have Accuracy of 60.4% with Decision Tree Model

Naive Bayes Classifier

In [0]:
#Importing NaiveBayes Classifier
from pyspark.ml.classification import NaiveBayes

In [0]:
#Naive Bayes Classifier
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol="labelIndex", featuresCol="final_features")

In [0]:
#Training my data on Naive Bayes Classifier
nbModel = nb.fit(train)
predictions = nbModel.transform(test)

In [0]:
#Made use of MulticlassClassificationEvaluator class in python to check the accuracy of the model.
eva = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = eva.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.33901 


We have Accuracy of 33.9% for Naive Bayes Classifier

The best Model for my Dataset is Decision Tree Classifier with the accuracy of 60.4%